In [ ]:
from torchvision import datasets
from torchvision import models
from torchvision import transforms
from torch import optim, cuda
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchsummary import summary
import seaborn as sns
import torch.nn as nn
import torch
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import pandas as pd
import warnings
import random
warnings.filterwarnings('ignore', category=FutureWarning)
from PIL import Image

In [ ]:
train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

In [ ]:
index_to_char_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing', 28: 'space'}

In [ ]:
#Parameters
dataset_1_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train' #Link: https://www.kaggle.com/grassknoted/asl-alphabet
dataset_2_dir = '../input/aslalphabettestmodified/asl-alphabet-test' #Link: https://www.kaggle.com/yashudaykulkarni/aslalphabettestmodified

img_dir = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/'

save_file_name_1 = 'vgg_cur_model_1.pt'
save_file_name_2 = 'vgg_cur_model_2.pt'

batch_size = 16
lr = 0.001
momentum = 0.9
epoch_stop_condition = 5
n_epochs = 5

dataset_1_length = 87000
dataset_2_length = 870

n_classes = 29

train_split_ratio = 0.5
val_split_ratio = 0.25

In [ ]:
image_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  
    ]),
    'val':
    transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 6)
        self.conv3 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 15 * 15, 480)
        self.fc2 = nn.Linear(480, 240)
        self.fc3 = nn.Linear(240, 29)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 15 * 15)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def dataloader_lengths(dataset_length, train_split_ratio, val_split_ratio):
    train_split = int(np.floor(train_split_ratio * dataset_length))
    val_split = int(np.floor(val_split_ratio * dataset_length))
    
    train_dataloader_length = train_split
    val_dataloader_length = val_split
    test_dataloader_length = dataset_length-(train_dataloader_length + val_dataloader_length)
    
    return train_dataloader_length, test_dataloader_length, val_dataloader_length

In [ ]:
def split_dataset(dataset_length, train_split_length, val_split_length, dataset):
    indices = list(range(dataset_length))
        
    np.random.shuffle(indices)

    train_indices = indices[:train_split_length]
    val_indices = indices[train_split_length:train_split_length+val_split_length]
    test_indices = indices[train_split_length+val_split_length:] 
    
    train_indices_df = pd.DataFrame(train_indices)
    train_indices_df.to_csv('train_indices_' + str(dataset) + '_vals.csv', index=False)
    
    test_indices_df = pd.DataFrame(test_indices)
    test_indices_df.to_csv('test_indices_' + str(dataset) + '_vals.csv', index=False)
    
    val_indices_df = pd.DataFrame(val_indices)
    val_indices_df.to_csv('val_indices_' + str(dataset) + '_vals.csv', index=False)

    return train_indices, test_indices, val_indices

In [ ]:
def create_ImageFolder(dataset_dir):
    data = {
        'train': datasets.ImageFolder(root=dataset_dir, transform=image_transforms['train']),
        'val': datasets.ImageFolder(root=dataset_dir, transform=image_transforms['val']),
        'test': datasets.ImageFolder(root=dataset_dir, transform=image_transforms['test']),
    }
    
    return data

In [ ]:
def create_sampler(train_indices, test_indices, val_indices):
    train_sampler = SubsetRandomSampler(train_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    
    return train_sampler, test_sampler, val_sampler

In [ ]:
def create_DataLoader(data, train_sampler, test_sampler, val_sampler):
    dataloader = {
        'train': DataLoader(data['train'], batch_size=batch_size, sampler=train_sampler),
        'val': DataLoader(data['val'], batch_size=batch_size, sampler=val_sampler),
        'test': DataLoader(data['test'], batch_size=batch_size, sampler=test_sampler),
    }
    
    return dataloader

In [ ]:
def get_model():
    model = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    if train_on_gpu:
        model = model.to('cuda')
    return model, criterion, optimizer

In [ ]:
def save_model(model):
    torch.save(model.state_dict(), save_file_name)

In [ ]:
def load_model(model):
    model.load_state_dict(torch.load(save_file_name))
    model.optimizer = optimizer
    if train_on_gpu:
        model = model.to('cuda')

In [ ]:
def train_model(model, train_dataloader, train_dataloader_length, val_dataloader, val_dataloader_length, criterion, optimizer, epoch_stop_condition, n_epochs):
    n_epochs_no_improvement = 0
    val_loss_min = np.Inf
    
    history = []
    
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')
    
    begin_time = timer()
    
    for epoch in range(n_epochs):
        cur_epoch_train_loss = 0.0
        cur_epoch_val_loss = 0.0
        
        cur_epoch_train_acc = 0
        cur_epoch_val_acc = 0
        
        model.train()
        cur_epoch_start_time = timer()
        
        for step, (features, labels) in enumerate(train_dataloader):
            if train_on_gpu:
                features = features.cuda()
                labels = labels.cuda()
                
            optimizer.zero_grad()

            output = model(features)

            loss = criterion(output, labels)
            loss.backward()

            optimizer.step()

            cur_epoch_train_loss += loss.item() * features.size(0)

            _, pred = torch.max(output, dim=1)
            cur_epoch_train_acc += torch.sum(pred == labels.data)
            
            torch.cuda.empty_cache()
            del features, labels, output, pred

            print(f'Epoch: {epoch}\t{100 * (step + 1) / len(train_dataloader):.2f}% complete. {timer() - cur_epoch_start_time:.2f} seconds elapsed in epoch.',end='\r')
                    
        
        model.epochs += 1
        
        with torch.no_grad():
            model.eval()
            
            for features, labels in val_dataloader:
                if train_on_gpu:
                    features = features.cuda()
                    labels = labels.cuda()

                output = model(features)

                loss = criterion(output, labels)

                cur_epoch_val_loss += loss.item() * features.size(0)

                _, pred = torch.max(output, dim=1)
                cur_epoch_val_acc += torch.sum(pred == labels.data)

                torch.cuda.empty_cache()
                del features, labels, output, pred
            
            cur_epoch_train_loss = cur_epoch_train_loss / train_dataloader_length
            cur_epoch_val_loss = cur_epoch_val_loss / val_dataloader_length
            
            cur_epoch_train_acc = cur_epoch_train_acc.double() / train_dataloader_length
            cur_epoch_val_acc = cur_epoch_val_acc.double() / val_dataloader_length
            
            history.append([cur_epoch_train_loss, cur_epoch_val_loss, cur_epoch_train_acc, cur_epoch_val_acc])
            
            print(f'\nEpoch: {epoch} \tTraining Loss: {cur_epoch_train_loss:.4f} \tValidation Loss: {cur_epoch_val_loss:.4f}')
            print(f'\t\tTraining Accuracy: {100 * cur_epoch_train_acc:.2f}%\t Validation Accuracy: {100 * cur_epoch_val_acc:.2f}%')
            
            if cur_epoch_val_loss < val_loss_min:
                torch.save(model.state_dict(), save_file_name)
                
                n_epochs_no_improvement = 0
                
                val_loss_min = cur_epoch_val_loss
                best_val_acc = cur_epoch_val_acc
                best_epoch = epoch
                
            else:
                n_epochs_no_improvement += 1
                
                if n_epochs_no_improvement >= epoch_stop_condition:
                    print(f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {val_loss_min:.2f} and acc: {100 * best_val_acc:.2f}%')
                    
                    total_time = timer() - begin_time
                    print(f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.')    
                
                    model.load_state_dict(torch.load(save_file_name))
                    model.optimizer = optimizer
                    
                    history = pd.DataFrame(history, columns=['train_loss', 'val_loss', 'train_acc','val_acc'])
                    
                    return model, history
    
    model.optimizer = optimizer
    
    total_time = timer() - begin_time
    
    print(f'\nBest epoch: {best_epoch} with loss: {val_loss_min:.2f} and acc: {100 * best_val_acc:.2f}%')
    print(f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.')    
              
    history = pd.DataFrame(history, columns=['train_loss', 'val_loss', 'train_acc','val_acc'])
                    
    return model, history

In [ ]:
def generate_freq_class_dataloader(dataloader):
    freq_list = np.zeros(29)

    for step, (images, labels) in enumerate(dataloader):
        for label in labels:
            freq_list[label] += 1
        del images, labels

    return freq_list

In [ ]:
def eval_model(test_dataloader, test_dataloader_length, freq_list):
    test_loss = 0.0
    test_acc = 0.0
    
    acc_every_class = np.zeros(29)
        
    for features, labels in test_dataloader:
        model.eval()
        
        if train_on_gpu:
            features = features.cuda()
            labels = labels.cuda()
            
        output = model(features)
        
        loss = criterion(output, labels)
        
        test_loss += loss.item() * features.size(0)
        
        _, pred = torch.max(output, dim=1)
        test_acc += torch.sum(pred == labels.data)
        
        for i, (p, label) in enumerate(zip(pred, labels)):
            acc_every_class[label] += (pred[i] == labels[i])
            
        torch.cuda.empty_cache()
        del features, labels, output, pred
        
    test_acc = test_acc.double() / test_dataloader_length
    test_loss = test_loss / test_dataloader_length
    
    for i in range(len(acc_every_class)):
        acc_every_class[i] = acc_every_class[i] / freq_list[i]
        
    return test_acc, test_loss, acc_every_class

In [ ]:
train_dataloader_1_length, test_dataloader_1_length, val_dataloader_1_length = dataloader_lengths(dataset_1_length, train_split_ratio, val_split_ratio)

In [ ]:
train_dataloader_2_length, test_dataloader_2_length, val_dataloader_2_length = dataloader_lengths(dataset_2_length, train_split_ratio, val_split_ratio)

In [ ]:
train_indices_1, test_indices_1, val_indices_1 = split_dataset(dataset_1_length, train_dataloader_1_length, val_dataloader_1_length, 1)

In [ ]:
train_indices_2, test_indices_2, val_indices_2 = split_dataset(dataset_2_length, train_dataloader_2_length, val_dataloader_2_length, 2)

In [ ]:
train_sampler_1, test_sampler_1, val_sampler_1 = create_sampler(train_indices_1, test_indices_1, val_indices_1)

In [ ]:
train_sampler_2, test_sampler_2, val_sampler_2 = create_sampler(train_indices_2, test_indices_2, val_indices_2)

In [ ]:
data_1 = create_ImageFolder(dataset_1_dir)

In [ ]:
data_2 = create_ImageFolder(dataset_2_dir)

In [ ]:
dataloader_1 = create_DataLoader(data_1, train_sampler_1, test_sampler_1, val_sampler_1)

In [ ]:
dataloader_2 = create_DataLoader(data_2, train_sampler_2, test_sampler_2, val_sampler_2)

In [ ]:
dummy_iter = iter(dataloader_1['train'])
features, labels = next(dummy_iter)
features.shape, labels.shape

In [ ]:
dummy_iter = iter(dataloader_2['train'])
features, labels = next(dummy_iter)
features.shape, labels.shape

In [ ]:
model, criterion, optimizer = get_model(pretrained=True)

In [ ]:
model, history = train_model(model, dataloader_1['train'], train_dataloader_1_length, dataloader_1['val'], val_dataloader_1_length, criterion, optimizer, epoch_stop_condition, n_epochs, save_file_name_1)

In [ ]:
history.to_csv(r'model_training_stats_1.csv')

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_loss', 'val_loss']:
    plt.plot(history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Cross Entropy Loss')
plt.title('Training and Validation Losses')

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_acc', 'val_acc']:
    plt.plot(100 * history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Accuracy')
plt.title('Training and Validation Accuracy')

In [ ]:
freq_list_test_1 = generate_freq_class_dataloader(dataloader_1['test'])

In [ ]:
test_acc_1, test_loss_1, acc_every_class_1 = eval_model(dataloader_1['test'], test_dataloader_1_length, freq_list_test_1)

In [ ]:
print(test_acc_1.item() * 100, test_loss_1)

In [ ]:
print(acc_every_class_1 * 100)

In [ ]:
freq_list_test_2 = generate_freq_class_dataloader(dataloader_2['test'])

In [ ]:
test_acc_2, test_loss_2, acc_every_class_2 = eval_model(dataloader_2['test'], test_dataloader_2_length, freq_list_test_2)

In [ ]:
print(test_acc_2.item() * 100, test_loss_2)

In [ ]:
print(acc_every_class_2 * 100)

In [ ]:
model, history = train_model(model, dataloader_2['train'], train_dataloader_2_length, dataloader_2['val'], val_dataloader_2_length, criterion, optimizer, epoch_stop_condition, n_epochs, save_file_name_2)

In [ ]:
history.to_csv(r'model_training_stats_2.csv')

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_loss', 'val_loss']:
    plt.plot(history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Cross Entropy Loss')
plt.title('Training and Validation Losses')

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_acc', 'val_acc']:
    plt.plot(100 * history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Accuracy')
plt.title('Training and Validation Accuracy')

In [ ]:
test_acc_2, test_loss_2, acc_every_class_2 = eval_model(dataloader_2['test'], test_dataloader_2_length, freq_list_test_2)

In [ ]:
print(test_acc_2.item() * 100, test_loss_2)

In [ ]:
print(acc_every_class_2 * 100)